In [1]:
# notebook playground for bigger dataset
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import albumentations as A
from torchvision.io import read_image


In [ ]:
# google docstring template
"""
    A class representing an employee.
 
    Attributes:
        name (str): The name of the employee.
        age (int): The age of the employee.
        department (str): The department the employee works in.
        salary (float): The salary of the employee.
    """

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
classes_labels = ['bicycle', 'bus', 'car', 'motorbike', 'person']

In [2]:
# Dataset on the Traffic Detection Dataset
# <class> <x_center> <y_center> <width> <height>
class VtecDataset():
    """Traffic Detection Dataset"""
    
    def __init__(self, img_dir, label_dir, transform=None, target_transform=None):
        """
        Attributes:
            img_dir (string): Path to training image directory
            label_dir (string): Path to training label directory
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.target_transform = target_transform
        self.images = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, i):
        """Returns subset of data and targets corresponding to given indices."""
        image_name = self.images[i]
        image_path = os.path.join(self.img_dir, image_name)
        image = read_image(image_path)
        
        label_name = image_name.replace('.jpg', '.txt')
        label_path = os.path.join(self.label_dir, label_name)
        labels = self.get_labels(label_path)
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            labels = self.target_transform(labels)
        
        return image, labels
    
    
    def get_labels(self, label_path):
        labels = []
        """Loads labels for corresponding image"""
        with open(label_path, "r") as f:
            lines = f.readlines()
        
        for line in lines:
            labels.append(line)
            
        return labels